In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula as smf
import datetime

vac = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto80/vacunacion_comuna_1eraDosis_std.csv')
idh = pd.read_csv('https://raw.githubusercontent.com/leoferres/gitiele356b/main/data/hdi.csv')
covidd = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna_std.csv')

In [3]:
vacc = vac['Codigo comuna'].isin(idh['comuna'])
vacc =vac.loc[vacc == True]
vacc

,Region,Codigo region,Comuna,Codigo comuna,Fecha,Primera Dosis
88,Metropolitana,13,Alhue,13502.0,2020-12-24,0.0
89,Metropolitana,13,Buin,13402.0,2020-12-24,0.0
90,Metropolitana,13,Calera de Tango,13403.0,2020-12-24,0.0
91,Metropolitana,13,Cerrillos,13102.0,2020-12-24,0.0
92,Metropolitana,13,Cerro Navia,13103.0,2020-12-24,0.0
...,...,...,...,...,...,...
72173,Metropolitana,13,San Ramon,13131.0,2021-07-11,196.0
72174,Metropolitana,13,Santiago,13101.0,2021-07-11,0.0
72175,Metropolitana,13,Talagante,13601.0,2021-07-11,0.0
72176,Metropolitana,13,Tiltil,13303.0,2021-07-11,0.0


In [9]:
vacc.dtypes

Region                   object
Codigo region             int64
Comuna                   object
Codigo comuna           float64
Fecha            datetime64[ns]
Primera Dosis           float64
dtype: object

Se cambia el tipo de dato de la columna fecha para poder operarlo en el formato fecha

In [8]:
vacc['Fecha'] = pd.to_datetime(vacc['Fecha'], dayfirst = True)

<ipython-input-8-7ba03a876c6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc['Fecha'] = pd.to_datetime(vacc['Fecha'], dayfirst = True)


In [11]:
grouper = pd.Grouper(key='Fecha', freq='W')
vacc["Semana"] = vacc.groupby(grouper).grouper.group_info[0] + 1
vacc = vacc.groupby(['Semana', 'Codigo comuna']).sum()

<ipython-input-11-8247c86fbcf8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc["Semana"] = vacc.groupby(grouper).grouper.group_info[0] + 1


In [13]:
vacc

Codigo region  Primera Dosis
Semana Codigo comuna                              
1      13101.0                   52         1228.0
       13102.0                   52            0.0
       13103.0                   52            0.0
       13104.0                   52            0.0
       13105.0                   52            0.0
...                             ...            ...
29     13601.0                   91         1306.0
       13602.0                   91          450.0
       13603.0                   91          478.0
       13604.0                   91         1379.0
       13605.0                   91         1404.0

[1508 rows x 2 columns]

In [15]:
vacc.drop('Codigo region', axis = 1)

Primera Dosis
Semana Codigo comuna               
1      13101.0               1228.0
       13102.0                  0.0
       13103.0                  0.0
       13104.0                  0.0
       13105.0                  0.0
...                             ...
29     13601.0               1306.0
       13602.0                450.0
       13603.0                478.0
       13604.0               1379.0
       13605.0               1404.0

[1508 rows x 1 columns]

In [18]:
covidd['Fecha'] = pd.to_datetime(covidd['Fecha'], dayfirst = True)
covidd

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Fecha,Casos fallecidos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-06-12,10.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-06-12,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-06-12,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-06-12,0.0
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,2020-06-12,NaN
...,...,...,...,...,...,...,...
42331,Magallanes y la Antartica,12,San Gregorio,12104.0,681.0,2021-07-09,1.0
42332,Magallanes y la Antartica,12,Timaukel,12303.0,282.0,2021-07-09,0.0
42333,Magallanes y la Antartica,12,Torres del Paine,12402.0,1021.0,2021-07-09,0.0
42334,Magallanes y la Antartica,12,Desconocido Magallanes,NaN,NaN,2021-07-09,0.0


In [20]:
covidscl = covidd['Codigo comuna'].isin(idh['comuna'])
covidscl = covidd.loc[covidscl == True]
covidscl.reset_index().drop('index', axis=1)

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Fecha,Casos fallecidos
0,Metropolitana,13,Alhue,13502.0,7405.0,2020-06-12,0.0
1,Metropolitana,13,Buin,13402.0,109641.0,2020-06-12,24.0
2,Metropolitana,13,Calera de Tango,13403.0,28525.0,2020-06-12,0.0
3,Metropolitana,13,Cerrillos,13102.0,88956.0,2020-06-12,19.0
4,Metropolitana,13,Cerro Navia,13103.0,142465.0,2020-06-12,77.0
...,...,...,...,...,...,...,...
5819,Metropolitana,13,San Ramon,13131.0,86510.0,2021-07-09,384.0
5820,Metropolitana,13,Santiago,13101.0,503147.0,2021-07-09,659.0
5821,Metropolitana,13,Talagante,13601.0,81838.0,2021-07-09,170.0
5822,Metropolitana,13,Tiltil,13303.0,21477.0,2021-07-09,43.0


In [ ]:
covidscl